介绍一些维度变换，线代矩阵运算的知识

In [1]:
import torch
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
B = A.clone()  # 通过分配新内存，将A的一个副本分配给B
x = torch.arange(4, dtype=torch.float32)
A, B,x

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]),
 tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]),
 tensor([0., 1., 2., 3.]))

降维  
计算其元素的和sum  
mean或average  

In [2]:
x.shape,A.shape,x.sum(), A.sum()#降为标量

(torch.Size([4]), torch.Size([5, 4]), tensor(6.), tensor(190.))

指定张量沿哪一个轴来通过求和降低一个维度，即指定的维度会被压成一维消失掉

In [3]:
A_sum_axis0 = A.sum(axis=0)
A_sum_axis0, A_sum_axis0.shape
#轴0表示行，现在只有一行了，变成了行向量（1,4）

(tensor([40., 45., 50., 55.]), torch.Size([4]))

In [4]:
A_sum_axis1 = A.sum(axis=1)
A_sum_axis1, A_sum_axis1.shape
#轴1表示列，现在只有一列了，变成了列向量（5,1）

(tensor([ 6., 22., 38., 54., 70.]), torch.Size([5]))

非降维求和  
广播机制要求维度相同，标量除外  
keepdims保持维度

In [5]:
sum_A = A.sum(axis=1, keepdims=True)
sum_A,sum_A.shape

(tensor([[ 6.],
         [22.],
         [38.],
         [54.],
         [70.]]),
 torch.Size([5, 1]))

In [6]:
A / sum_A

tensor([[0.0000, 0.1667, 0.3333, 0.5000],
        [0.1818, 0.2273, 0.2727, 0.3182],
        [0.2105, 0.2368, 0.2632, 0.2895],
        [0.2222, 0.2407, 0.2593, 0.2778],
        [0.2286, 0.2429, 0.2571, 0.2714]])

线性代数矩阵运算  
向量点积  

In [7]:
y = torch.ones(4, dtype = torch.float32)
x, y, torch.dot(x, y)

(tensor([0., 1., 2., 3.]), tensor([1., 1., 1., 1.]), tensor(6.))

In [8]:
torch.sum(x * y)

tensor(6.)

矩阵与向量相乘  
把矩阵看作一个列向量，列向量的每一行都是行向量  
行向量与被乘的向量作点积  
要求矩阵列数与向量size(也是行向量)相同

In [9]:
A.shape, x.shape, torch.mv(A, x),torch.mv(A, x).shape

(torch.Size([5, 4]),
 torch.Size([4]),
 tensor([ 14.,  38.,  62.,  86., 110.]),
 torch.Size([5]))

矩阵与矩阵的乘法  
要求前面的列等于后面的行  
可以把A看作列向量$\begin{bmatrix} a1\\a2\\...\\am\end{bmatrix}$，B看作行向量$\begin{bmatrix} b1&b2&...&bn\end{bmatrix}$  
$C={\left\{a_i \cdot b_j \right\}}_{m \times n}$  

In [10]:
B = torch.ones(4, 3)
C=torch.mm(A, B)
print(C)
print(A.shape,B.shape,C.shape)

tensor([[ 6.,  6.,  6.],
        [22., 22., 22.],
        [38., 38., 38.],
        [54., 54., 54.],
        [70., 70., 70.]])
torch.Size([5, 4]) torch.Size([4, 3]) torch.Size([5, 3])


### 批量矩阵乘法
假设第一个小批量数据包含$n$个矩阵$\mathbf{X}_1,\ldots, \mathbf{X}_n$，
形状为$a\times b$，
第二个小批量包含$n$个矩阵$\mathbf{Y}_1, \ldots, \mathbf{Y}_n$，
形状为$b\times c$。  
它们的批量矩阵乘法得到$n$个矩阵
$\mathbf{X}_1\mathbf{Y}_1, \ldots, \mathbf{X}_n\mathbf{Y}_n$，
形状为$a\times c$。  
因此，[**假定两个张量的形状分别是$(n,a,b)$和$(n,b,c)$，
它们的批量矩阵乘法输出的形状为$(n,a,c)$**]。

In [11]:
X = torch.ones((2, 1, 4))
Y = torch.ones((2, 4, 6))
torch.bmm(X, Y).shape

torch.Size([2, 1, 6])

向量范数计算  
距离，二范数  
L1范数，绝对值再求和  
还有P范数，(sum(abs(xi)^p))^(1/p) 需要自己搭建

In [12]:
u = torch.tensor([3.0, -4.0])
torch.norm(u)

tensor(5.)

In [13]:
torch.abs(u).sum()#L1范数

tensor(7.)

矩阵的Frobenius范数  
矩阵元素平方和的平方根

In [14]:
torch.norm(torch.ones((4, 9)))

tensor(6.)

repeat_interleave 是 PyTorch 中用于按元素重复张量的方法，将 input 中的每个元素连续重复 repeats 次  
torch.repeat_interleave(input, repeats, dim=None) → Tensor  
    input：输入张量  
    repeats：每个元素重复的次数（可以是整数或张量），连续重复  
    dim：沿着哪个维度重复（如果为None，会先展平）  
    
A.repeat_interleave(repeats, dim=None)

In [15]:
x = torch.tensor([1, 2, 3])
# repeat_interleave: 按元素连续重复
result_interleave = torch.repeat_interleave(x, 2)
print("repeat_interleave(x, 2):", result_interleave)

repeat_interleave(x, 2): tensor([1, 1, 2, 2, 3, 3])


按指定次数重复每个维度
tensor.repeat(*sizes) → Tensor  
*sizes：每个维度重复的次数（可以是一个整数元组或单独的参数）  
 一维张量X_(m,)使用 repeat((n, h)) 时，PyTorch 会：  
 1. 首先在 X 前面添加一个维度，变成 (1, m) , x_expanded = X.unsqueeze(0)
 2. 然后应用 repeat((n, h)) , 形状: (n, m*h) , X_tile = x_expanded.repeat((n, h))  

不是连续重复，整个维度重复

In [16]:
# repeat: 按维度重复整个张量
result_repeat = x.repeat(2)
print("x.repeat(2):", result_repeat)

x.repeat(2): tensor([1, 2, 3, 1, 2, 3])


In [17]:
result_repeat = x.repeat((2,2))
print(result_repeat)

tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])


torch.max函数在指定维度上操作时，会返回一个包含两个张量的元组（namedtuple）：

第一个张量是最大值本身。  
第二个张量是这些最大值对应的索引

In [19]:
X = torch.randn((2, 4))
(Y,index)=X.max(1)
X,Y,index

(tensor([[-0.1543, -1.3496, -1.2837, -0.8476],
         [-1.3621,  0.8969,  1.0386,  0.4951]]),
 tensor([-0.1543,  1.0386]),
 tensor([0, 2]))